In [3]:
!pip -q install sagemaker pandas awscli --upgrade

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install boto3 --upgrade

     |████████████████████████████████| 131 kB 16.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 51.6 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 934 kB/s s eta 0:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.112
    Uninstalling botocore-1.20.112:
      Successfully uninstalled botocore-1.20.112
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.4.2
    Uninstalling s3transfer-0.4.2:
      Successfully uninstalled s3transfer-0.4.2
  Attempting uninstall: boto3
    Found existing installation: boto3 1.17.99
    Uninstalling boto3-1.17.99:
      Successfully uninstalled boto3-1.17.99
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.19.112 requires botocore==1.20.112, but you have botocore 1.21.8 which is incompatible.
awscli 1.19.112 requires s3tra

In [1]:
import boto3
import sagemaker

print(sagemaker.__version__)
session = sagemaker.Session()
bucket = session.default_bucket()

%env bucket s3://$bucket

2.45.0
env: bucket=s3://sagemaker-ap-northeast-1-906756132937


In [2]:
region_name = boto3.Session().region_name

algorithm = sagemaker.amazon.amazon_estimator.get_image_uri(
    region_name, "image-classification", "latest")

print("Using algorithm %s" % algorithm)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


Using algorithm 501404015308.dkr.ecr.ap-northeast-1.amazonaws.com/image-classification:1


In [6]:
!wget http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec
!wget http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec

--2021-07-28 10:21:30--  http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec
Resolving data.mxnet.io (data.mxnet.io)... 172.217.25.115, 2404:6800:4004:813::2013
Connecting to data.mxnet.io (data.mxnet.io)|172.217.25.115|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://data.mxnet.io.s3-website-us-west-1.amazonaws.com/data/caltech-256/caltech-256-60-train.rec [following]
--2021-07-28 10:21:30--  http://data.mxnet.io.s3-website-us-west-1.amazonaws.com/data/caltech-256/caltech-256-60-train.rec
Resolving data.mxnet.io.s3-website-us-west-1.amazonaws.com (data.mxnet.io.s3-website-us-west-1.amazonaws.com)... 52.219.113.35
Connecting to data.mxnet.io.s3-website-us-west-1.amazonaws.com (data.mxnet.io.s3-website-us-west-1.amazonaws.com)|52.219.113.35|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 363622688 (347M) [binary/octet-stream]
Saving to: ‘caltech-256-60-train.rec’

caltech-256-60-trai 100%[===============

In [3]:
session.upload_data(path='caltech-256-60-train.rec', bucket=bucket, key_prefix='train')
session.upload_data(path='caltech-256-60-val.rec',   bucket=bucket, key_prefix='validation')

s3_train      = 's3://{}/train/'.format(bucket)
s3_validation = 's3://{}/validation/'.format(bucket)
s3_output     = 's3://{}/output'.format(bucket)

%env s3_train      $s3_train
%env s3_validation $s3_validation
%env s3_output     $s3_output

env: s3_train=s3://sagemaker-ap-northeast-1-906756132937/train/
env: s3_validation=s3://sagemaker-ap-northeast-1-906756132937/validation/
env: s3_output=s3://sagemaker-ap-northeast-1-906756132937/output


In [4]:
train_data = sagemaker.session.s3_input(s3_train, 
                                    distribution='FullyReplicated',        
                                    content_type='application/x-recordio', 
                                    s3_data_type='S3Prefix')

validation_data = sagemaker.session.s3_input(s3_validation,
                                             distribution='FullyReplicated', 
                                             content_type='application/x-recordio', 
                                             s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [5]:
ic = sagemaker.estimator.Estimator(algorithm,
                                   sagemaker.get_execution_role(), 
                                   train_instance_count=1, 
                                   train_instance_type='ml.p2.xlarge',
                                   output_path=s3_output,
                                   sagemaker_session=session)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [15]:
ic.set_hyperparameters(num_layers=18,               # Train a Resnet-18 model
                       use_pretrained_model=1,      # Fine-tune on our dataset
                       image_shape="3,224,224",   # 3 channels (RGB), 224x224 pixels
                       num_classes=257,             # 256 classes + 1 clutter class
                       num_training_samples=15420,  # Number of training samples
                       mini_batch_size=128,
                       epochs=1,                  # Learn the training samples 1 times
                       learning_rate=0.1)

In [16]:
ic.fit(inputs=data_channels, logs=True)

2021-07-28 11:18:28 Starting - Starting the training job...
2021-07-28 11:18:30 Starting - Launching requested ML instancesProfilerReport-1627471108: InProgress
......
2021-07-28 11:19:44 Starting - Preparing the instances for training...............
2021-07-28 11:22:24 Downloading - Downloading input data...
2021-07-28 11:22:45 Training - Downloading the training image..Docker entrypoint called with argument(s): train
[07/28/2021 11:23:09 INFO 140679680079680] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/image_classification/default-input.json: {'use_pretrained_model': 0, 'num_layers': 152, 'epochs': 30, 'learning_rate': 0.1, 'lr_scheduler_factor': 0.1, 'optimizer': 'sgd', 'momentum': 0, 'weight_decay': 0.0001, 'beta_1': 0.9, 'beta_2': 0.999, 'eps': 1e-08, 'gamma': 0.9, 'mini_batch_size': 32, 'image_shape': '3,224,224', 'precision_dtype': 'float32'}
[07/28/2021 11:23:09 INFO 140679680079680] Merging with provided configuration from /opt/ml/input/config/hy

In [17]:
ic_endpoint_ei = ic.deploy(initial_instance_count=1,
                         instance_type='ml.p2.xlarge',        
                         accelerator_type='ml.eia2.xlarge',  
                         endpoint_name='ic-endpoint-ei')

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateEndpoint operation: The account-level service limit 'Number of elastic inference accelerators across active endpoints' is 0 Accelerators, with current utilization of 0 Accelerators and a request delta of 1 Accelerators. Please contact AWS support to request an increase for this limit.